In [24]:
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Visualizing
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

import acquire_PJ


# Sklearn Cluster and Scaler 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Tests RFE, REGRESSION MODEL
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
import sklearn.preprocessing


# Other Models
from sklearn.linear_model import LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

import scipy.stats as stats
from scipy.stats import pearsonr, spearmanr


# rmse to check baseline and model
from sklearn.metrics import mean_squared_error

In [2]:
zillow_df = acquire_PJ.get_zillow_data()
zillow_df.head()

,Unnamed: 0,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [3]:
# Create a function that will remove rows and columns that have missing values past a certain threshold.
def handle_missing_values(df, p_row = 0.84, p_col = 0.84):
    ''' function which takes in a dataframe, required notnull proportions of non-null rows and columns.
    drop the columns and rows columns based on theshold:'''
    
    #drop columns with nulls
    threshold = int(p_col * len(df.index)) # Require that many non-NA values.
    df.dropna(axis = 1, thresh = threshold, inplace = True)
    
    #drop rows with nulls
    threshold = int(p_row * len(df.columns)) # Require that many non-NA values.
    df.dropna(axis = 0, thresh = threshold, inplace = True)
    
    
    return df

In [4]:
zillow_df = handle_missing_values(zillow_df)

In [5]:
# Create a list of columns to drop.
columns_to_drop = ['transactiondate','assessmentyear','id','Unnamed: 0','parcelid','calculatedbathnbr'
                   ,'finishedsquarefeet12','fullbathcnt','propertycountylandusecode','propertylandusetypeid',
                   'rawcensustractandblock','regionidcity','regionidcounty','regionidzip',
                   'structuretaxvaluedollarcnt','censustractandblock','propertylandusedesc', 'roomcnt']



In [6]:
def drop_columns(df, drop_col):
    df = df.drop(columns=drop_col)
    return df

In [7]:
zillow_df = drop_columns(zillow_df, columns_to_drop)

In [8]:
for col in zillow_df.columns:
    if zillow_df[col].isna().sum() > 0:
        zillow_df[col] = zillow_df[col].fillna(value = zillow_df[col].mean())
        #print(zillow_df[col])

In [9]:
zillow_df = zillow_df.rename(columns = {'fips':'county', 'calculatedfinishedsquarefeet' : 'area', 'bathroomcnt' : 'bathrooms',
                         'bedroomcnt' : 'bedrooms', 'poolcnt' : 'pools', 'garagecarcnt' : 'garages',
                          'taxvaluedollarcnt': 'tax_value'})

In [10]:
# Cleaning the data
# Creating counties
zillow_df['LA_County']= zillow_df['county'] == 6037
zillow_df['Orange_County']= zillow_df['county'] == 6059
zillow_df['Ventura_County']= zillow_df['county'] == 6111


zillow_df['LA_County'] = zillow_df['LA_County'].replace(False, 0)
zillow_df['LA_County'] = zillow_df['LA_County'].replace(True, 1)

zillow_df['Orange_County'] = zillow_df['Orange_County'].replace(False, 0)
zillow_df['Orange_County'] = zillow_df['Orange_County'].replace(True, 1)

zillow_df['Ventura_County'] = zillow_df['Ventura_County'].replace(False, 0)
zillow_df['Ventura_County'] = zillow_df['Ventura_County'].replace(True, 1)

In [11]:
def yearbuilt_years(df):
    df.yearbuilt =  df.yearbuilt.astype(int)
    year = date.today().year
    df['age'] = year - df.yearbuilt
    # dropping the 'yearbuilt' column now that i have the age
    df = df.drop(columns=['yearbuilt'])
    return df

In [12]:
zillow_df = yearbuilt_years(zillow_df)

In [13]:
zillow_df = zillow_df[zillow_df.bedrooms != 0]
zillow_df = zillow_df[zillow_df.bathrooms != 0]

In [14]:
# Later Lets look at pools and garages!
# Create new features here

zillow_df['acres']= zillow_df.lotsizesquarefeet/43560


# ratio of bathrooms to bedrooms
zillow_df['bed_bath_ratio'] = zillow_df.bedrooms / zillow_df.bathrooms

# bin ages
zillow_df['age_bins'] = pd.cut(zillow_df.age, 
                               bins = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140],
                               labels = [0, .066, .133, .20, .266, .333, .40, .466, .533, 
                                     .60, .666, .733, .8, .866, .933])

# bin acres
zillow_df['acres_bin'] = pd.cut(zillow_df.acres, bins = [0, .10, .15, .25, .5, 1, 5, 10, 20, 50, 200], 
                    labels = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9])
    

#zillow_df['logerror_bins'] = pd.cut(zillow_df.logerror, bins = [-5, -3, -2, -1, -.05, 0, .05, 1, 2, 3, 4],
#                                      labels = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9])    

# update datatypes of binned values to be float
#zillow_df = zillow_df.astype({'acres_bin': 'float64', 'age_bin': 'float64'}).dtypes
zillow_df = zillow_df.astype({'acres_bin': float, 'age_bins': float}) #'logerror_bins' : float

In [15]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''

    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [17]:
col_list = ['bedrooms', 'bathrooms', 'area', 'tax_value',
            'lotsizesquarefeet', 'taxamount', 'age', 'acres',
           'bed_bath_ratio', 'landtaxvaluedollarcnt']
k = 1.5
    
zillow_df = remove_outliers(zillow_df, k, col_list)

In [18]:
# Splitting my data
def split(df, target_var):
    
    # split df into train_validate (80%) and test (20%)
    train_validate, test = train_test_split(df, test_size=.20, random_state=13)
    
    # split train_validate into train(70% of 80% = 56%) and validate (30% of 80% = 24%)
    train, validate = train_test_split(train_validate, test_size=.3, random_state=13)

    # create X_train by dropping the target variable 
    X_train = train.drop(columns=[target_var])
    # create y_train by keeping only the target variable.
    y_train = train[[target_var]]

    # create X_validate by dropping the target variable 
    X_validate = validate.drop(columns=[target_var])
    # create y_validate by keeping only the target variable.
    y_validate = validate[[target_var]]

    # create X_test by dropping the target variable 
    X_test = test.drop(columns=[target_var])
    # create y_test by keeping only the target variable.
    y_test = test[[target_var]]

    partitions = [train, X_train, y_train, validate, X_validate, y_validate, test, X_test, y_test]
    return partitions

In [19]:
partitions = split(zillow_df, target_var = 'logerror')

# Scaling Stuff

In [33]:
X_train = partitions[1]

# Dropping Unneeded features
X_train = train.drop(columns = ['county', 'bathrooms', 'bedrooms', 'latitude', 'longitude',
                               'lotsizesquarefeet', 'landtaxvaluedollarcnt', 'taxamount',
                               'LA_County', 'Orange_County', 'Ventura_County', 'bed_bath_ratio',
                                'age_bins', 'acres_bin', 'logerror'])


# Set scaler
scaler = MinMaxScaler(copy=True)
x_train_scaled = scaler.fit_transform(X_train)

x_train_scaled = pd.DataFrame(x_train_scaled, columns = X_train.columns.to_list())

In [34]:
X_train.head()

,area,tax_value,age,acres
24406,2305.0,745639.0,97,0.159389
59332,1691.0,132910.0,68,0.167080
28755,1716.0,710000.0,55,0.165289
31277,1862.0,425139.0,60,0.243802
14046,1692.0,250981.0,74,0.180785


# Creating base model

In [21]:
train = partitions[0]
train.head()

,bathrooms,bedrooms,area,county,latitude,longitude,lotsizesquarefeet,tax_value,landtaxvaluedollarcnt,taxamount,logerror,LA_County,Orange_County,Ventura_County,age,acres,bed_bath_ratio,age_bins,acres_bin
24406,3.0,5.0,2305.0,6037.0,34075447.0,-118134733.0,6943.0,745639.0,447300.0,8719.67,-0.051110,1,0,0,97,0.159389,1.666667,0.666,0.2
59332,2.0,3.0,1691.0,6037.0,34138129.0,-118458502.0,7278.0,132910.0,58999.0,1727.31,-0.114010,1,0,0,68,0.167080,1.500000,0.466,0.2
28755,2.5,3.0,1716.0,6059.0,33699842.0,-117962388.0,7200.0,710000.0,612583.0,7872.34,-0.018472,0,1,0,55,0.165289,1.200000,0.400,0.2
31277,2.0,4.0,1862.0,6059.0,33751576.0,-117824705.0,10620.0,425139.0,323707.0,5017.62,0.051606,0,1,0,60,0.243802,2.000000,0.400,0.2
14046,3.0,5.0,1692.0,6037.0,34275588.0,-118449462.0,7875.0,250981.0,95552.0,3077.32,-0.166924,1,0,0,74,0.180785,1.666667,0.533,0.2


In [35]:
y_train = partitions[2]
# Setting y_train to a data frame


y_train = pd.DataFrame(y_train)

y_train['baseline_mean'] = y_train['logerror'].mean()
y_train['baseline_med'] = y_train['logerror'].median()


y_train.head()

,logerror,baseline_mean,baseline_med,tax_pred_lm
24406,-0.051110,0.015679,0.005487,0.009642
59332,-0.114010,0.015679,0.005487,0.020146
28755,-0.018472,0.015679,0.005487,0.015275
31277,0.051606,0.015679,0.005487,0.023467
14046,-0.166924,0.015679,0.005487,0.022955


In [41]:
rmse_train = mean_squared_error(y_train.logerror, y_train.baseline_mean)**(1/2)
rmse_train2 = mean_squared_error(y_train.logerror, y_train.baseline_med)**(1/2)

print("RMSE using Mean \n Train / In-Sample", round(rmse_train, 5))
print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train2, 3))

RMSE using Mean 
 Train / In-Sample 0.16444
RMSE using Median
Train/In-Sample:  0.165


# Using Models
## OLS MODEL 1 

In [37]:
# create the model object
lm = LinearRegression(normalize=True)

# fit the model to our training data.
# since we have converted it to a dataframe from a series! 
lm.fit(x_train_scaled, y_train.logerror)

# predict train
y_train['tax_pred_lm'] = lm.predict(x_train_scaled)

# evaluate: rmse
rmse_train_ols = mean_squared_error(y_train.logerror, y_train.tax_pred_lm)**(1/2)


print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train_ols)

RMSE for OLS using LinearRegression
Training/In-Sample:  0.16427466848815073


## Laso

In [38]:
lars = LassoLars(alpha=1.0, random_state=123)

# fit the model to our training data. 
# since we have converted it to a dataframe from a series! 
lars.fit(x_train_scaled, y_train.logerror)

# predict train
y_train['tax_pred_lars'] = lars.predict(x_train_scaled)

# evaluate: rmse
rmse_train_lars = mean_squared_error(y_train.logerror, y_train.tax_pred_lars)**(1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train_lars) 

RMSE for Lasso + Lars
Training/In-Sample:  0.16443995174606943


# Polynomial

In [39]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(x_train_scaled)

In [40]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data.
# since we have converted it to a dataframe from a series! 
lm2.fit(x_train_scaled, y_train.logerror)

# predict train
y_train['tax_pred_lm2'] = lm2.predict(x_train_scaled)

# evaluate: rmse
rmse_train_poly = mean_squared_error(y_train.logerror, y_train.tax_pred_lm2)**(1/2)

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train_poly)

RMSE for Polynomial Model, degrees=2
Training/In-Sample:  0.16427466848815073
